In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense ,Dropout
import glob
import keras
from sklearn.preprocessing import LabelEncoder
import tensorflow_io as tfio
from scipy.io import wavfile as wav


In [ ]:
#label all of the audio files in train1 directory except the (ALL , test) folder

files = glob.glob('train1/*/*')
files1 = glob.glob('train1/*\\')
print(files)
l2 = []
for i, f in enumerate (files1):
    
    if f.find('All') > -1 or f.find('test') > -1:
        continue;
        
    print(f)
    l2.append(f)
print(files1[0].replace('\\' ,  '/'))
print(l2)
l2
df = pd.DataFrame()
l =[]
l3 = []
for i, f in enumerate (files):
    if f.find('All') > -1 or f.find('test') > -1:
            continue
    n=f
    for x in l2:
        n = n.replace(x , '')
        
    shaikh = f.replace('train1' , '')
    shaikh = shaikh.replace(n , '')
    shaikh = shaikh.replace('\\' , '')
    print(n)
    print(shaikh)
    l.append(n)
    l3.append(shaikh)

df['fileName'] = l
df['shaikh'] = l3
df

In [ ]:
df['shaikh'].unique()

In [ ]:

df

In [ ]:
# Extracting features from an audio file
def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=50)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features
    

In [ ]:
from tqdm import tqdm
# Now we iterate through every audio file and extract features 
# using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    # We will put a copy of all of the audio files in a folder called (All) to reach it easily and use our dataframe to lable it
    file_name = 'train1/All/'+str(row['fileName'])
    final_class_labels=row["shaikh"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df
#audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

In [ ]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [ ]:
X.shape


In [ ]:
###for RNN
X2 = np.expand_dims(X, 1) 

In [ ]:
X2.shape

In [ ]:
X

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
pd.get_dummies(y)

In [ ]:
#lb = LabelEncoder()
#y = lb.fit_transform(y)
y=np.array(pd.get_dummies(y))


In [ ]:
print(y[311])

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X2_train,X2_test,y_train,y_test=train_test_split(X2,y,test_size=0.2,random_state=0)

In [ ]:
X_train


In [ ]:
#sc = MinMaxScaler(feature_range=(-1 , 1))
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [ ]:
X_train

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
num_labels=y.shape[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,LSTM,GRU,SimpleRNN,Embedding ,Masking
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
###ANN
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.6))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.6))

###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.6))



###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 50*3

num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1 )


duration = datetime.now() - start
print("Training completed in time: ", duration)


In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

In [ ]:
filename="testsudais3.wav"
prediction_feature=features_extractor(filename)
prediction_feature=prediction_feature.reshape(1,-1)
model.predict(prediction_feature)

In [ ]:
###RNN
model2=Sequential()


###first layer
model2.add(LSTM(100, input_shape=(X2.shape[1:]) , return_sequences=True))
model2.add(Activation('relu'))
model2.add(Dropout(0.65))
###second layer
model2.add(LSTM(200 ,  return_sequences=True))
model2.add(Activation('relu'))
model2.add(Dropout(0.65))

###third layer
model2.add(LSTM(100 ,  return_sequences=True))
model2.add(Activation('relu'))
model2.add(Dropout(0.65))

model2.add(LSTM(50 ,  return_sequences=False))
model2.add(Activation('relu'))
model2.add(Dropout(0.65))

###final layer
model2.add(Dense(num_labels))
model2.add(Activation('softmax'))

In [ ]:
model2.summary()

In [ ]:
model2.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:


num_epochs = 100

num_batch_size = 32

checkpointer2 = ModelCheckpoint(filepath='saved_models/audio_classification2.hdf5', 
                               verbose=1, save_best_only=True)
start2 = datetime.now()

model2.fit(X2_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X2_test, y_test), callbacks=[checkpointer2], verbose=1 )


duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
filename="testsaad3.wav"
prediction_feature=features_extractor(filename)
prediction_feature=prediction_feature.reshape(1 , 1 , 50)
model2.predict(prediction_feature)

In [ ]:
prediction_feature.shape = (1, 1, 50)